In [9]:
from torrent.peer import Peer, Peer_manager, _valid_peer
from torrent.tracker import tracker_manager
from torrent.torrent_file import generate_torrent_file
from torrent.pieces import create_piece_manager
import asyncio
import logging
PATH = '../test_files/file3.torrent'
BLOCK_SIZE = 2 ** 11
torrent_file = generate_torrent_file(PATH)
logger = logging.getLogger(__name__)

In [2]:
peer_manager = Peer_manager()
tracker_threads = tracker_manager(torrent_file, peer_manager)
logger = logging.getLogger(__name__)
piece_manager = create_piece_manager(torrent_file)


2020-04-07 19:36:53,117 - torrent.tracker - INFO - 294 peers add to peer_manager and go to sleep for 1800 seconds	


# peers

In [3]:
from urllib.parse import urlparse
from torrent.bencoding import Decoder, Encoder
from random import randint, choice
import logging
import struct
import string
from torrent.utilities import run_async
import socket


In [4]:
def handshake(torrent_file):
    const_protocol = 'BitTorrent protocol'.encode('utf-8')
    info_hash = torrent_file.info_hash
    peer_id = torrent_file.id.encode('utf-8')
    return struct.pack('>B19s8x20s20s', 19, const_protocol, info_hash, peer_id)



In [5]:
def get_message_type(res):
    """
        False empty 
        -1 - keep alive
        0 - choke
        1 - unchoke
        2 - interested
        3 - uninterested
        4 - have
        5 - bitfield (what i got my brother)
        6 - request
        7 - piece
        8 - cancel
    """
    #empty
    if res == b'':
        logger.info('we got empty response')
        return None, None
    #keep alive
    elif len(res) == 4 and res == b'0000':
        return -1, 0
    #message type
    elif res[4] in range(0,8):
        parsed_res = struct.unpack(">IB", res)
        if not parsed_res:
            logger.info("parsed_res got broken")
            return None, None
        return parsed_res
    else:
        logger.info(f"we got for response {res[4]}")
        return None, None
    #empty
    if res == b'':
        logger.info('we got empty response')
        return None, None
    #keep alive
    elif len(res) == 4 and res == b'0000':
        return -1, 0
    #message type
    elif res[4] in range(0,8):
        parsed_res = struct.unpack(">IB", res)
        if not parsed_res:
            logger.info("parsed_res got broken")
            return None, None
        return parsed_res
    else:
        logger.info(f"we got for response {res[4]}")
        return None, None



def valid_res_hand_shake(res):
    if len(res) != 68:
        logger.info('you didnt got handshake')
        return False
    else:
        return True



In [10]:
async def peering(torrent_file, piece_manager, peer):
    #connection and hand shake
    try:
        reader, writer = await asyncio.open_connection(peer.ip, peer.port)
    except Exception as e:
        logger.info(exc_info=True)
        return False
    logging.info('connection pass')
    writer.write(handshake(torrent_file))
    logging.info('handshake send')
    res = valid_res_hand_shake(await reader.read(68))
    if not res:
        logger.info('handshake peer failed')
        return False
    logger.info('handshake succeeded')
    
    #bitfield
    size, message_type = get_message_type(await reader.read(5))
    if message_type == 5:
        bitfiled_data = await reader.read(size + 10)
        if len(bitfiled_data) + 1 != size:
            #TODO handle check what pieces peer has
            logger.info(f"bitfiled size is {size} and we got {len(bitfiled_data)}")
    else:
        logger.info(f"in bitfield we got '{message_type}' insted of 5")
    
    #Interested
    writer.write(interested_message())
    size, message_type = get_message_type(await reader.read(5))
    if message_type == 0:
        logger.info('peer choke')
        return False
    elif message_type == None:
        return False
    elif message_type == 1:
        logger.info('peer unchoke')
    #pieces
    while True:
        piece = await piece_manager.get_piece()
        offset = 0
        counter = 0
        while piece.length>= offset:
            writer.write(request_message(piece.index, offset, BLOCK_SIZE))
            meta_message = await reader.read(5)
            if not meta_message:
                logger.info(f"we got nothing from request in round - {counter}")
                piece.lost_piece()
                break
            size, message_type = get_message_type(meta_message) 
            if message_type != 7:
                logger.info(f"in request we got '{message_type}' insted of 7")
                piece.lost_piece()
                break
            if not size or not message_type:
                logger.info(f"somthing get worng in {counter} request {meta_message} got from peer")
            
            #chunks
            res = await reader.read(size)
            if res == b'':
                logger.info("we got empty data")
                break
            
            chunk_index, chunk_offset = struct.unpack(f">II", res[0:8])
            offset += BLOCK_SIZE
            piece.add_block(res[8:len(res)])
            counter += 1
        piece.piece_done()
        
    
    


In [11]:
async def runn(torrent_file, piece_manager, peer_manager):
    await asyncio.gather(*[peering(torrent_file, piece_manager, peer) for peer in peer_manager._peer_list[3:6]])
    

In [ ]:
await runn(torrent_file, piece_manager, peer_manager)

2020-04-07 19:39:16,433 - root - INFO - 8 connection pass	
2020-04-07 19:39:16,453 - root - INFO - 10 handshake send	
2020-04-07 19:39:16,465 - root - INFO - 8 connection pass	
2020-04-07 19:39:16,482 - root - INFO - 10 handshake send	
2020-04-07 19:39:16,494 - root - INFO - 8 connection pass	
2020-04-07 19:39:16,516 - root - INFO - 10 handshake send	
2020-04-07 19:39:16,848 - __main__ - INFO - 15 handshake succeeded	
2020-04-07 19:39:16,851 - __main__ - INFO - 15 handshake succeeded	
2020-04-07 19:39:17,374 - __main__ - INFO - 15 handshake succeeded	
2020-04-07 19:39:17,546 - __main__ - INFO - 36 peer unchoke	
2020-04-07 19:39:17,554 - __main__ - INFO - 36 peer unchoke	
2020-04-07 19:39:17,878 - __main__ - INFO - 31 we got for response 164	
2020-04-07 19:39:17,892 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:17,895 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:17,911 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2

2020-04-07 19:39:19,206 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:19,222 - __main__ - INFO - 31 we got for response 111	
2020-04-07 19:39:19,238 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:19,247 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:19,258 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:19,284 - __main__ - INFO - 31 we got for response 133	
2020-04-07 19:39:19,287 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:19,309 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:19,327 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:19,348 - __main__ - INFO - 31 we got for response 27	
2020-04-07 19:39:19,377 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:19,392 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:19,397 - torrent.pieces - I

2020-04-07 19:39:20,485 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:20,493 - __main__ - INFO - 31 we got for response 45	
2020-04-07 19:39:20,500 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:20,504 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:20,508 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:20,517 - __main__ - INFO - 31 we got for response 192	
2020-04-07 19:39:20,524 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:20,531 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:20,532 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:20,534 - __main__ - INFO - 31 we got for response 37	
2020-04-07 19:39:20,540 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:20,547 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:20,548 - torrent.pieces - IN

2020-04-07 19:39:21,053 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:21,053 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:21,065 - __main__ - INFO - 31 we got for response 15	
2020-04-07 19:39:21,068 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:21,069 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:21,076 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:21,077 - __main__ - INFO - 31 we got for response 216	
2020-04-07 19:39:21,085 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:21,093 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:21,104 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:21,105 - __main__ - INFO - 31 we got for response 255	
2020-04-07 19:39:21,109 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:21,126 - torrent.pieces - I

2020-04-07 19:39:21,561 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:21,564 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:21,565 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:21,578 - __main__ - INFO - 31 we got for response 61	
2020-04-07 19:39:21,583 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:21,584 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:21,588 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:21,590 - __main__ - INFO - 31 we got for response 219	
2020-04-07 19:39:21,599 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:21,601 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:21,601 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:21,603 - __main__ - INFO - 31 we got for response 39	
2020-04-07 19:39:21,616 - __main__ - INFO - 5

...

2020-04-07 19:39:29,678 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:29,678 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:29,684 - __main__ - INFO - 31 we got for response 92	
2020-04-07 19:39:29,685 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:29,695 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:29,696 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:29,698 - __main__ - INFO - 31 we got for response 93	
2020-04-07 19:39:29,698 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:29,704 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:29,705 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:29,712 - __main__ - INFO - 31 we got for response 221	
2020-04-07 19:39:29,719 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:29,721 - torrent.pieces - IN

2020-04-07 19:39:30,194 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:30,196 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:30,202 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:30,211 - __main__ - INFO - 31 we got for response 41	
2020-04-07 19:39:30,213 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:30,213 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:30,214 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:30,219 - __main__ - INFO - 31 we got for response 251	
2020-04-07 19:39:30,221 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:30,228 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:30,229 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:30,233 - __main__ - INFO - 31 we got for response 229	
2020-04-07 19:39:30,244 - __main__ - INFO - 

2020-04-07 19:39:30,740 - __main__ - INFO - 31 we got for response 119	
2020-04-07 19:39:30,741 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:30,743 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:30,744 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:30,753 - __main__ - INFO - 31 we got for response 129	
2020-04-07 19:39:30,756 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:30,771 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:30,772 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:30,774 - __main__ - INFO - 31 we got for response 95	
2020-04-07 19:39:30,784 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:30,787 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:30,788 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:30,796 - __main__ - INFO - 

2020-04-07 19:39:31,289 - __main__ - INFO - 31 we got for response 186	
2020-04-07 19:39:31,294 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:31,303 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:31,305 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:31,306 - __main__ - INFO - 31 we got for response 90	
2020-04-07 19:39:31,315 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:31,317 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:31,318 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:31,322 - __main__ - INFO - 31 we got for response 252	
2020-04-07 19:39:31,333 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:31,336 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:31,341 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:31,349 - __main__ - INFO - 

2020-04-07 19:39:31,977 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:31,984 - __main__ - INFO - 31 we got for response 212	
2020-04-07 19:39:31,992 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:31,993 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:32,000 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:32,018 - __main__ - INFO - 31 we got for response 221	
2020-04-07 19:39:32,019 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:32,027 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:32,036 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:32,038 - __main__ - INFO - 31 we got for response 204	
2020-04-07 19:39:32,044 - __main__ - INFO - 51 in request we got 'None' insted of 7	
2020-04-07 19:39:32,045 - torrent.pieces - INFO - 36 we lost a piece in index 2	
2020-04-07 19:39:32,053 - torrent.pieces - 

In [13]:
piece_manager

piece_manager{<PieceStatus.free: 0>: 2317, <PieceStatus.done: 2>: 0, <PieceStatus.in_progress: 1>: 33}